<a href="https://colab.research.google.com/github/tpenanen/Laptop-Price-Project/blob/main/Computer_Data_Set_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Group Names: Yasseen, Aditya, Manbhav, Tanmay

For our project, we have decided to use the "Laptop Prices" dataset. This dataset describes specific details about laptops on the current market such as their brands, operating systems, specifications, as well as their price (In Euros originally until we added a USD column). For this data, we are studying and trying to predict the pricing of the laptop by using the main features of specs (just a generalized term that describes the overall specs of the computer) as well as company in order to see who charges a specific value based on components so we can properly predict the future pricing of any new laptops that come from the company.

UPDATE: We are currently trying to figure out how to encode our categorical features so we can use a correlation matrix in order to see how corrrelated each component is to price.

In [75]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score

data = pd.read_csv("laptop_price.csv", encoding='latin-1')
data.head()

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros,Price_usd
0,6,Acer,Aspire 3,Notebook,15.6,1366x768,AMD A9-Series 9420 3GHz,4GB,500GB HDD,AMD Radeon R5,Windows 10,2.1kg,400.0,436.7600
1,10,Acer,Swift 3,Ultrabook,14.0,IPS Panel Full HD 1920x1080,Intel Core i5 8250U 1.6GHz,8GB,256GB SSD,Intel UHD Graphics 620,Windows 10,1.6kg,770.0,840.7630
2,37,Acer,Aspire 3,Notebook,15.6,1366x768,Intel Core i3 7130U 2.7GHz,4GB,1TB HDD,Intel HD Graphics 620,Linux,2.1kg,367.0,400.7273
3,44,Acer,Aspire A515-51G,Notebook,15.6,IPS Panel Full HD 1920x1080,Intel Core i5 8250U 1.6GHz,4GB,256GB SSD,Intel UHD Graphics 620,Windows 10,2.2kg,682.0,744.6758
4,52,Acer,Aspire A515-51G,Notebook,15.6,IPS Panel Full HD 1920x1080,Intel Core i7 8550U 1.8GHz,8GB,256GB SSD,Nvidia GeForce MX150,Windows 10,2.2kg,841.0,918.2879


In [76]:
data['target'] = (data['Price_usd']).astype(int)
del data['Price_usd']

In [77]:
categorical_columns = data.select_dtypes(include='object').columns.tolist()
print(categorical_columns)

['Company', 'Product', 'TypeName', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight']


The following is to one-hot encode the categorical features through a pipeline.

This is to help set up the train test split.

In [78]:
data_features = data.drop(["target", "Price_euros"], axis=1)

In order to solve the case of human error, we have imputed the median into any numeric feature that needs it. Also, for cat_feat, we are currently trying to figure out how to one-hot encode our categorical features in order to prepare for the matrix.

In [79]:
numfeat = ["Inches"]
catfeat = ["Company", "Product", "TypeName", "ScreenResolution", "Cpu", "Ram", "Memory", "Gpu", "OpSys", "Weight"]

num_pipeline = Pipeline([("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())])

cat_pipeline = Pipeline([("onehot", OneHotEncoder(handle_unknown="ignore"))])

full_pipeline = ColumnTransformer([("num", num_pipeline, numfeat), ("cat", cat_pipeline, catfeat)])

We are now checking to see which company's mean price for all laptops on the market are the most expensive and the most cheap.

In [80]:
data.groupby("Company")["target"].mean().sort_values(ascending=False)

,target
Company,
Razer,3653.142857
LG,2291.333333
MSI,1887.277778
Google,1831.666667
Microsoft,1760.000000
Apple,1707.428571
Huawei,1554.000000
Samsung,1542.777778
Toshiba,1383.812500


Now we will need to use a correlation matrix to see which values we want to drop (Which are irrelevant?)

In [81]:
datafeatpp = full_pipeline.fit_transform(data_features)
display(datafeatpp)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 14333 stored elements and shape (1303, 1148)>

In [82]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

x = datafeatpp
y = data["target"].values

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(xtrain, ytrain)

print(f"R-squared score: {r2_score(ytest, model.predict(xtest)):.2f}")

R-squared score: 0.76


In [86]:
corr_matrix = datafeatpp.corr(numeric_only = False)

AttributeError: 'csr_matrix' object has no attribute 'corr'